<h1>Titanic exercise</h1>

Load libraries

In [133]:
import pandas as pd
from sklearn.cluster import KMeans #The k-means algorithm

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

Open file

In [134]:
df = pd.read_csv("titanic.csv")
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Cabin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,C85
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,C123
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,NaN
...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,NaN
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,B42
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,NaN
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,C148


Select columns 

In [1]:
dfSubset = df[["Survived", "Pclass", "Sex", "Age", "SibSp"]]

dfSubset    
dfSubset.info()

# dfSubset["Survived"].value_counts()

NameError: name 'df' is not defined

2. Now get rid of the rows with empty cells.

In [136]:
dfSubset = dfSubset.dropna()
dfSubset

# Erik: no normalisation here, because all values are small. Age is not ranging from 0 to 3000 infinite whatever now. We have a lot of ones and zeros now in the rest of the columns. If the majority of the dataset would differ a lot, then normalising would be smart.

,Survived,Pclass,Sex,Age,SibSp
0,0,3,male,22.0,1
1,1,1,female,38.0,1
2,1,3,female,26.0,0
3,1,1,female,35.0,1
4,0,3,male,35.0,0
...,...,...,...,...,...
885,0,3,female,39.0,0
886,0,2,male,27.0,0
887,1,1,female,19.0,0
889,1,1,male,26.0,0


3. Add dummy variables for the categorical variable.

In [137]:
dummies = pd.get_dummies(dfSubset["Sex"])
dfSubset = pd.concat([dfSubset, dummies], axis=1) #the axis=1 means: add it to the columns (axis=0 is rows)
dfSubset

,Survived,Pclass,Sex,Age,SibSp,female,male
0,0,3,male,22.0,1,0,1
1,1,1,female,38.0,1,1,0
2,1,3,female,26.0,0,1,0
3,1,1,female,35.0,1,1,0
4,0,3,male,35.0,0,0,1
...,...,...,...,...,...,...,...
885,0,3,female,39.0,0,1,0
886,0,2,male,27.0,0,0,1
887,1,1,female,19.0,0,1,0
889,1,1,male,26.0,0,0,1


4. Split the data into training and test data.

In [138]:
x = dfSubset[["Pclass", "Age", "SibSp", "female", "male"]]
y = dfSubset["Survived"]

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1) #split the data, store it into different variables

X_train

,Pclass,Age,SibSp,female,male
641,1,24.0,0,1,0
433,3,17.0,0,0,1
202,3,34.0,0,0,1
585,1,18.0,0,1,0
544,1,50.0,1,0,1
...,...,...,...,...,...
179,3,36.0,0,0,1
808,2,39.0,0,0,1
93,3,26.0,1,0,1
291,1,19.0,1,1,0


5. Train a kNN-algorithm on the data, with k = 3

In [139]:
from sklearn.neighbors import KNeighborsClassifier #the object class we need

knn = KNeighborsClassifier(n_neighbors=3) #create a KNN-classifier with 3 neighbors
knn = knn.fit(X_train, y_train) #this fits the k-nearest neigbor model with the train data

calculate knn

In [140]:
knn.score(X_test, y_test) #calculate the fit on the test data

0.8232558139534883

0.8 knn score

In [141]:
from sklearn.metrics import confusion_matrix
y_test_pred = knn.predict(X_test) #the predicted values
cm = confusion_matrix(y_test, y_test_pred) #creates a "confusion matrix"
cm

array([[114,  20],
       [ 18,  63]], dtype=int64)

5.5. Calculate the recall for survival on the test set.

In [142]:
conf_matrix = pd.DataFrame(cm, index=["Not survived (actual)", "Survived (actual)"], columns = ["Not survived (predicted)", "Survived (predicted)"]) #make a dataframe, put labels on rows (index) and columns 
conf_matrix

,Not survived (predicted),Survived (predicted)
Not survived (actual),114,20
Survived (actual),18,63
